# 浮動小数点数

## 浮動小数点数 (Floating Point Number)
計算機において小数は，**浮動小数点数**として実装されている．


In [2]:
0.0000123

1.23e-5

浮動小数点数の標準規格はIEEE754で策定されており，次のような要素で構成される．
- 正規化数 (normal number)
- 非正規化数 (subnormal number)
- 特殊な値：`Inf` （無限大） や `NaN` (Not a Number) など．


## 単精度と倍精度

多くの計算機でハードウェアレベルで実装されている浮動小数点数は
 - 単精度 (single precision) ：2進数23桁
 - 倍精度 (double precision) ：2進数52桁
 
の２種類である．違いは表現可能な実数の範囲だけであり，倍精度の方が表現できる実数の範囲が広い． 

Juliaでは `Float64`が倍精度，`Float32`が単精度，に対応している．
 
小数の数値リテラルは，暗黙のうちに `Float64` と解釈される．


In [3]:
typeof(0.1)

Float64

特別な理由がない限り，普通は倍精度を用いる．

これ以降，倍精度を具体例として浮動小数点数の説明を述べる．

## 正規化数，非正規化数
正規化数とは，`1.0101e-5` のように先頭桁が1の浮動小数点数である．

仮数部は2進数表示である．

- $d_1, d_2, \ldots, d_{52} \in \{0, 1\}$
- $-1022 \le e \le 1023$
    
非正規化数は，$0.d_1 d_2 \cdots d_{52} \times 2^{-1023}$の形で表される小数のことである．
$d_1,...d_{52}$は正規化数と同様に 0 or 1 の値を取る．
<br>

正規化数を数直線上にプロットすると原点付近で相対的に大きな隙間が生まれる．
非正規化数はこの隙間を均質に埋める役割を担っている．

<img src = "fpn01.png" width="80%">

## E表記 `e`, `E`
倍精度浮動小数点数では， `e` あるいは `E` の後ろに指数を表記する．

In [2]:
0.000000123

1.23e-7

In [5]:
0.12345e+6  

123450.0

## Overflow
倍精度で扱える最大の正の正規化数は`floatmax(Float64)`で取得できる．

In [7]:
x_max = floatmax(Float64)

1.7976931348623157e308

浮動小数点数の演算の結果が `x_max`を超えた場合，無限大を表す `Inf` に変換される．
このような現象を overflow という．

In [8]:
2 * x_max

Inf

#### Note
`x_max`は $1.11\cdots 1_2 \times 2^{1023}$に等しい．
添字の2は2進数を表す．

`BigFloat`で計算して一致することを確認する．
仮数部を10進数に直すと，
$$
2^{0} + 2^{-1} + \cdots + 2^{-53} = \frac{1 - 2^{-54}}{1- 2^{-1}}
 = 2 - 2^{-54}.
$$


In [9]:
(2 - big(2.0)^(-54)) * big(2.0)^1023

1.797693134862315857833297452421029442208659153021342421048019300690888220491329e+308

確かに一致している．

#### Note
倍精度で扱える *最小の正の正規化数* を知りたい場合は，`floatmin`を使う．

In [10]:
x_min = floatmin(Float64)

2.2250738585072014e-308

## Underflow
最小の正の非正規化数は `eps(0.0)` で取得できる．

In [11]:
subnormal_min = eps(0.0)

5.0e-324

（正しい値は $0.00\cdots 01_2 \times 2^{-1022} = 4.940\cdots \times 10^{-324}$なので，表示されている数値は誤りです．
内部的には正しい値が格納されているようなので，表示上のバグかもしれません．原因が分かる人は教えてください．）

In [12]:
bitstring(subnormal_min) # bit列の表示．これは問題ない．

"0000000000000000000000000000000000000000000000000000000000000001"

演算結果が この値を下回ると `0.0` に丸められることがあり，これを underflow という．

In [13]:
subnormal_min / 2

0.0

#### Note
`subnormal_min` は $0.00\cdots 01_2 \times 2^{-1022}$に等しい．
仮数部は10進数で表すと，$2^{-52}$だから，$2^{-(1022 + 52)} = 2^{-1074}$.
`BigFloat`で表示してみる．

In [14]:
big(2.0)^(-1074)

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

In [15]:
big(eps(0.0))   # BigFloatで確認．一致する．

4.940656458412465441765687928682213723650598026143247644255856825006755072702088e-324

## `NaN` (Not a Number)

浮動小数点数において未定義な演算等を行った結果，`NaN` という特殊な浮動小数点数が発生することがある．

例えば，0を0で割ると `Nan` が発生する．

In [16]:
0 / 0

NaN

ちなみに0以外の正の浮動小数点数をゼロで割ると`Inf`になる．

In [17]:
1 / 0

Inf

これ以外にも，`Inf`に関する四則演算の結果，`NaN` が発生することがある． 

In [18]:
@show Inf + Inf
@show Inf - Inf    # ∞ - ∞
@show Inf * Inf
@show Inf * 0      # ∞ ✕ 0
@show Inf / Inf    # ∞ ÷ ∞

Inf + Inf = Inf
Inf - Inf = NaN
Inf * Inf = Inf
Inf * 0 = NaN
Inf / Inf = NaN


NaN

不定形の極限に対応するような演算を行うと `NaN` が発生すると覚えておけばよい．

#### Warning
- `Inf` や `NaN` の発生を前提としたコード設計はしないようにしてください．
`Inf` や `NaN`の挙動を完全に把握することは非常に困難で，bug （プログラムにおける予期しない動作やエラー）
を生みやすいです．さらに，読み手にとっても理解しにくいコードになります．
- Juliaでは `isfinite(x)` で `x` が有限値であるかどうか， `isnan(x)`で `x` が `NaN` であるかどうかの判定ができます．
    